In [1]:
logdir = 'data_array/'
import numpy as np
x = np.random.random((2000, 2000))

In [2]:
import pickle
%time pickle.dump(x,open(logdir+'x.pkl','wb'),protocol=4)

CPU times: user 10.9 ms, sys: 40.4 ms, total: 51.4 ms
Wall time: 50.6 ms


In [24]:
import cloudpickle
%time cloudpickle.dump(x,open(logdir+'x.cloud_pkl','wb'),protocol=4)

CPU times: user 22.9 ms, sys: 44.6 ms, total: 67.5 ms
Wall time: 66.5 ms


In [24]:
import pickle
import gzip
def save_pickle(object, filename, protocol = pickle.HIGHEST_PROTOCOL):
        """Saves a compressed object to disk
        """
        fil = gzip.GzipFile(filename, 'wb')
        fil.write(pickle.dumps(object, protocol))
        fil.close()
def save_cPickle(object, filename, protocol = pickle.HIGHEST_PROTOCOL):
        """Saves a compressed object to disk
        """
        fil = gzip.GzipFile(filename, 'wb')
        fil.write(pickle.dumps(object, protocol))
        fil.close()

In [42]:
def save_pickle(object, filename, protocol = pickle.HIGHEST_PROTOCOL):
        """Saves a compressed object to disk
        """
        fil = gzip.GzipFile(logdir+'x.pklgz', 'wb')
        fil.write(pickle.dumps(object, protocol))
        fil.close()
%time save_pickle(x,logdir+'x.pklgz')

CPU times: user 1.12 s, sys: 35 ms, total: 1.15 s
Wall time: 1.15 s


In [31]:
import _pickle as cPickle
%time cPickle.dump(x,open(logdir+'x.cpkl','wb'),protocol=pickle.HIGHEST_PROTOCOL)

CPU times: user 10.7 ms, sys: 31.7 ms, total: 42.4 ms
Wall time: 41.8 ms


In [32]:
%time save_cPickle(x,logdir+'x.cpklgz')

CPU times: user 1.12 s, sys: 27.3 ms, total: 1.14 s
Wall time: 1.14 s


In [11]:
import pickle
import gzip
fil = gzip.GzipFile(logdir+'x.cpkl_gzip', 'wb')
%time fil.write(pickle.dumps(x, protocol=4))
fil.close()

CPU times: user 1.17 s, sys: 50.6 ms, total: 1.22 s
Wall time: 1.23 s


In [34]:
import hickle
%time hickle.dump(x,open(logdir+'x.hkl','w'), compression='gzip')

CPU times: user 933 ms, sys: 10.7 ms, total: 944 ms
Wall time: 942 ms


In [37]:
import hickle
%time hickle.dump(x,open(logdir+'x.hkl_zero','w'))

CPU times: user 1.34 ms, sys: 19.6 ms, total: 21 ms
Wall time: 20.4 ms


In [5]:
import hickle
k = hickle.load(logdir+'x.hkl')

In [1]:
from json_tricks import dump, dumps, load, loads, strip_comments
%time dump(x,open(logdir+'x.json','w'))

NameError: name 'x' is not defined

In [41]:
import h5py
with h5py.File(logdir+'x.h5_gzip', 'w') as f:
    def write(dsetname, a):
        f.create_dataset(dsetname, data=a, compression='gzip')
    %time write('x',x)

CPU times: user 938 ms, sys: 0 ns, total: 938 ms
Wall time: 935 ms


In [3]:
import h5py
f = h5py.File(logdir+'x.h5_raw', 'w')
%time f.create_dataset('x', data=x)
f.close

OSError: Unable to create file (unable to truncate a file which is already open)

In [9]:
import h5py
%time h5py.File(logdir+'x.h5_gzip', 'w').create_dataset('x', data=x, compression='gzip')

CPU times: user 920 ms, sys: 13.6 ms, total: 934 ms
Wall time: 931 ms


<HDF5 dataset "x": shape (2000, 2000), type "<f8">

In [10]:
import h5py
%time h5py.File(logdir+'x.h5_lzf', 'w').create_dataset('x', data=x, compression='lzf')

CPU times: user 137 ms, sys: 12 ms, total: 149 ms
Wall time: 148 ms


<HDF5 dataset "x": shape (2000, 2000), type "<f8">

In [40]:
import h5py
with h5py.File(logdir+'x.h5_lzf', 'w') as f:
    def write(dsetname, a):
        f.create_dataset(dsetname, data=a, compression='lzf')
    %time write('x',x)

CPU times: user 137 ms, sys: 12.1 ms, total: 149 ms
Wall time: 150 ms


In [12]:
import bcolz
%time bcolz.carray(x, rootdir=logdir+'bcolz', mode='w')

CPU times: user 94.1 ms, sys: 51.1 ms, total: 145 ms
Wall time: 93 ms


carray((2000, 2000), float64)
  nbytes := 30.52 MB; cbytes := 27.14 MB; ratio: 1.12
  cparams := cparams(clevel=5, shuffle=1, cname='lz4', quantize=0)
  chunklen := 32; chunksize: 512000; blocksize: 32768
  rootdir := 'data_array/bcolz'
  mode    := 'w'
[[ 0.4088873   0.02608459  0.13016727 ...,  0.18862415  0.30075478
   0.02447061]
 [ 0.34429628  0.84285643  0.62440176 ...,  0.25277555  0.49511973
   0.9347124 ]
 [ 0.24118754  0.76635816  0.89680104 ...,  0.92640556  0.35458828
   0.79862376]
 ..., 
 [ 0.73590475  0.60360785  0.0107433  ...,  0.15572024  0.41766816
   0.14112706]
 [ 0.02551004  0.39150602  0.16281551 ...,  0.16897539  0.96894141
   0.628273  ]
 [ 0.15050865  0.50612845  0.1214735  ...,  0.20838497  0.26733341
   0.25380589]]

In [6]:
import array_pb2

In [7]:
def load_protobuf(path):
    pb_data = array_pb2.NDArray()
    pb_data.ParseFromString(open(path, 'r').read())
    return np.array(pb_data.array, dtype=pb_data.dtype).reshape(tuple(pb_data.shape))

In [16]:
import array_pb2
parray = list(x.flatten().astype(int))
with open("x.pbf", "wb") as pbf:
    arr = array_pb2.NDArray()
    arr.array.extend(parray)
    arr.shape.extend(list(x.shape))
    arr.dtype = x.dtype.name
    %time pbf.write(arr.SerializeToString())

# with open("test2.pbf", "wb") as pbf:
#     arr = array_pb2.NDArrayPacked()
#     arr.array.extend(parray)
#     arr.shape.extend(list(x.shape))
#     arr.dtype = x.dtype.name
#     pbf.write(arr.SerializeToString())

# %time arr8 = load_protobuf("test1.pbf")
# %time arr8 = load_protobuf("test2.pbf")

CPU times: user 53.9 ms, sys: 16.4 ms, total: 70.3 ms
Wall time: 70.3 ms
